# Análise do Número de Venda dos Consoles de cada Empresa com base na Região

Nessa análise nosso intuito é saber como as diferentes regiões, com suas diferentes características de mercado, influenciam a venda de alguns consoles. Usaremos os dados disponíveis [Platform Totals - VGChartz](https://www.vgchartz.com/charts/platform_totals/Hardware.php). 

### Regiões

A plataforma coletou esses dados desde 2004 separando as vendas nas regiões de north_america (América do Norte), europe (Europa), japan (Japão) e other (Outros).

Filtramos para que possamos lidar somente com consoles que tenham números de vendas registrados para as 4 regiões.

### Empresas

As empresas de jogos com as quais lidaremos são: Atari, Microsoft, Nintendo, SEGA e Sony.

Atari, SEGA e Microsoft tem 4 ou menos consoles registrados cada, o que pode acabar gerando um viés maior em suas estatísticas, por representarem uma parcela bem menor do total.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from io import StringIO
from datetime import datetime

In [ ]:
consoles = ['PS5', 'XS', 'NS', 'PSV', 'XOne', 'PS4', 'WiiU','3DS', 'PS3', 'X360', 'Wii', 'PSP'] #except for WiiU
consoles.reverse()
last_console = df[df['console'] == 'DS'].set_index("week").rename(columns={"sales" : 'DS'}).get(['DS'])
for console in consoles:
    console_data = df[df['console'] == console].set_index("week").rename(columns={"sales" : console}).get([console])
    last_console = pd.concat([last_console,console_data], axis=1).fillna(0)
    
last_console.to_csv('../data/console_matrix.csv',index=True)
last_console

In [9]:
df_region = (pd.read_csv("../data/console_region.csv")).sort_values(by='company')
company = df_region.groupby('company').sum(numeric_only = True)

regions = ['north_america', 'europe', 'japan', 'other']
companies = ['Atari', 'Microsoft', 'Nintendo', 'SEGA', 'Sony']

In [8]:
regions = ['north_america', 'europe', 'japan', 'other']

for r in regions:
    company[r] = (company[r] / company['global']).round(3)

for r in regions:
    df_region[r] = (df_region[r] / df_region['global']).round(3)

company

,north_america,europe,japan,other,global
company,,,,,
Atari,0.785,0.112,0.079,0.025,30.00
Microsoft,0.564,0.276,0.015,0.146,201.07
Nintendo,0.418,0.274,0.238,0.070,861.28
SEGA,0.395,0.283,0.120,0.201,65.52
Sony,0.336,0.340,0.153,0.171,638.03
Total,0.411,0.295,0.176,0.118,1795.90


## Como testaremos

Queremos agora testar como cada região influencia a venda de cada empresa:

> Para tal vamos pegar o número de vendas da empresa E na região R e dividir pelas vendas totais da mesma empresa.
>
> Ao mesmo tempo, vamo pegar o número de vendas que não correspondem a empresa E na região R e dividir pelas vendas totais de todos os consoles, menos o da empresa E.
> 
> Faremos um teste de hipótese de comparação de proporçoes populacionais para verificar

### Teste de Hipótese
O teste de hipótese deve ser feito da seguinte forma:
> Temos como p1 a proporção de vendas na região R da empresa E, n1 é o número total de vendas da empresa E e m1 é o total de vendas da empresa E na região R. Dessa forma, p1 = m1/n1
>
> Temos também p2, a proporção de vendas na região R sem a empresa E, n1 o total de vendas sem a empresa E e m2 o total de vendas sem a empresa E na região R. De forma semelhante, p2 = m2/n2
>
> p0, ou p-linha na imagem, segue a fórmula de (m1 + m2)/(n1 + n2)
>
> O valor observado Z_obs, calculado como mostrado abaixo, tem distribuição normal-padrão - N(média = 0, dp = 1)

![](../images/teste_proporcao.png)

### Hipóteses
Compararemos cada proporção de forma que nossa que
> Hipótese Nula: p1 = p2
>
> Hipótese Alternativa: p1 != p2


No teste usaremos nível de α = 0,05

Ou seja, rejeitaremos a Hipótese Nula se o valor absoluto de Z_obs for maior que 1,96. Isso se deve ao fato de P(Z > 1,96) = 0,025, se juntarmos com P(Z < -1,96) = 0,025 temos α.

### Teste com a Nintendo no Japão
Vamos primeiramente analisar o comportamento dos consoles da Nintendo na região do Japão. Tradicionalmente espera-se que a região impacte positivamente as vendas, já que a empresa é da região e grande parte de seus fãs são japoneses.

Nesse caso específico usaremos
> Hipótese Nula: p1 <= p2
>
> Hipótese Alternativa: p1 > p2
>
> α = 0,05

Rejeitaremos a Hipótese Nula se Z_obs for maior que 1.645

In [11]:
nintendo = company.loc['Nintendo']
not_nintendo = company.loc['Total'] - nintendo
n1 = nintendo.get('global')
n2 = not_nintendo.get('global')
m1 = nintendo.get('japan')
m2 = not_nintendo.get('japan')
p1 = m1/n1
p2 = m2/n2
p0 = (m1 + m2)/(n1 + n2)
z = (p1 - p2) / math.sqrt(p0 * (1 - p0) * (1/n1 + 1/n2))
z.round(3)

6.649

### Resultado
> Z_obs = 6.649
>
> Z_obs > 1.645

Conclusão: Rejeitamos a Hipótese Nula de que a proporção de consoles da Nintendo vendidos no Japão em comparação com os que não são da Nintendo vendidos no Japão é menor ou igual.

### Continuação
Expandiremos o teste para todas as empresas com todas as regiões. Criaremos uma tabela para tal que conterá todos os Z_obs


In [13]:
matrix = []
for c in companies:
    vector = []
    for r in regions:
        in_company = company.loc[c]
        not_in_company = company.loc['Total'] - in_company
        n1 = in_company.get('global')
        n2 = not_in_company.get('global')
        m1 = in_company.get(r)
        m2 = not_in_company.get(r)
        p1 = m1/n1
        p2 = m2/n2
        p0 = (m1 + m2)/(n1 + n2)
        vector.append(((p1 - p2) / math.sqrt(p0 * (1 - p0) * (1/n1 + 1/n2))).round(3))
    matrix.append(vector)
matrix
proportions = pd.DataFrame(data=matrix, index=companies ,columns=regions)
proportions

,north_america,europe,japan,other
Atari,4.197,-2.225,-1.408,-1.593
Microsoft,4.688,-0.646,-6.372,1.280
Nintendo,0.623,-1.900,6.649,-6.103
SEGA,-0.261,-0.217,-1.201,2.120
Sony,-4.760,3.090,-1.895,5.124


### Conclusão Geral
Com base na tabela temos como conclusão:
> Atari: É afetada positivamente pela America do Norte e negativamente pela Europa
>
> Microsoft: É afetada positivamente pela America do Norte e negativamente pelo Japão
>
> Nintendo: É afetada positivamente pelo Japão e negativamente por "Outros"
>
> SEGA: É afetada positivamente por "Outros"
>
> Sony: É afetada negativamente pela America do Norte e positivamente pela Europa e por "Outros"

### Observações

> Região "Outros": Nintendo X Sony
>
> Microsoft: Japão x América do Norte
>
> Nintendo: Dominância no Japão

## Análise Profunda do Caso Nintendo no Japão
Com o teste de hipótese acima, é comum acreditar que somente com o nome da marca, os consoles da Nintendo vão vender melhor no Japão. Isso pode até ser verdade, mas não é garantido. Pode ser, por exemplo, que metade dos consoles da Nintendo venda MUITO melhor no Japão enquanto os outros fiquem na média ou ainda abaixo dela.

Para termos mais informações sobre esse caso e também um pouco sobre como funciona o mercado japonês, testaremos com cada um dos consoles da Nintendo.

Com isso também podemos achar algumas anomalias dentre os consoles da empresa

In [27]:
nintendo_consoles = df_region[(df_region.get("company") == "Nintendo")]
nintendo_consoles = nintendo_consoles.drop(columns=["company", "north_america", "europe", "other"])
nintendo_consoles = nintendo_consoles.set_index('console')
consoles = nintendo_consoles.index
nintendo_array = []
not_nintendo = company.loc['Total'] - company.loc['Nintendo']
for c in consoles:
    in_console = nintendo_consoles.loc[c]
    n1 = in_console.get('global')
    n2 = not_nintendo.get('global')
    m1 = in_console.get('japan')
    m2 = not_nintendo.get('japan')
    p1 = m1/n1
    p2 = m2/n2
    p0 = (m1 + m2)/(n1 + n2)
    nintendo_array.append(((p1 - p2) / math.sqrt(p0 * (1 - p0) * (1/n1 + 1/n2))).round(3))

nintendo_consoles = nintendo_consoles.assign(japan_z_obs=nintendo_array)
nintendo_consoles

,japan,global,japan_z_obs
console,,,
Nintendo 64,5.54,32.93,0.865
Super Nintendo Entertainment System,17.17,49.10,4.700
Nintendo 3DS,24.67,75.94,5.082
Game Boy Advance,16.96,81.51,2.344
Nintendo Entertainment System,19.35,61.91,4.394
Nintendo Wii,12.77,101.63,0.216
Nintendo Wii U,3.33,13.56,1.430
Game Boy,32.47,118.69,4.649
Nintendo Switch,35.58,150.25,3.946


### Conclusão Parcial
É interessante notar que todos os consoles da Nintendo tem uma uma mudança neutra ou positiva se estão no Japão.

Dos 11 consoles testados da empresa, 7 são influenciados positivamente pelo mercado japonês.

Isso indica com grande confiança que, só de ser da Nintendo os consoles tem grande chance de serem afetados positivamente em território japonês. Não se trata somente de um caso ridiculamente grande de estouro no Japão, é um padrão seguido pelos produtos da empresa.

Isso colabora com a ideia geral e popular de que a Nintendo tem uma grande legião fixa de fãs. Enquanto algumas franquias vem e vão, a Nintendo consolidou sua base e não parece querer abandoná-la.

### Nintendo x Nintendo
Nessa sessão vamos analisar cada console da Nintendo com os outros consoles da própria Nintendo. 

Dessa forma vamos identificar as anomalias (além dos que não são afetados pelo mercado japonês do caso acima) e tentar traçar um padrão entre elas.

In [29]:
nintendo_x_nintendo = []
for c in consoles:
    in_console = nintendo_consoles.loc[c]
    n1 = in_console.get('global')
    n2 = nintendo_consoles.get('global').sum() - n1
    m1 = in_console.get('japan')
    m2 = nintendo_consoles.get('japan').sum() - m1
    p1 = m1/n1
    p2 = m2/n2
    p0 = (m1 + m2)/(n1 + n2)
    nintendo_x_nintendo.append(((p1 - p2) / math.sqrt(p0 * (1 - p0) * (1/n1 + 1/n2))).round(3))
    
nintendo_consoles = nintendo_consoles.assign(nintendo_z_obs=nintendo_x_nintendo)
nintendo_consoles

,japan,global,japan_z_obs,nintendo_z_obs
console,,,,
Nintendo 64,5.54,32.93,0.865,-0.957
Super Nintendo Entertainment System,17.17,49.10,4.700,1.895
Nintendo 3DS,24.67,75.94,5.082,1.865
Game Boy Advance,16.96,81.51,2.344,-0.664
Nintendo Entertainment System,19.35,61.91,4.394,1.433
Nintendo Wii,12.77,101.63,0.216,-2.829
Nintendo Wii U,3.33,13.56,1.430,0.067
Game Boy,32.47,118.69,4.649,0.984
Nintendo Switch,35.58,150.25,3.946,-0.034


### Conclusão Final
Com a nova tabela, podemos tirar novas conclusões e hipóteses interessantes. 

Primeiramente, nenhum dos valores de console parece ser afetador mais positivamente pelo Japão do que a média da Nintendo em si. 

Apesar disso, podemos notar um que é afetado negativamente em comparação a média.

Esse console anômalo é o Wii. 

É um resultado que faz sentido ao pensar na febre que foi esse console, que vendeu bem até mesmo em países que a Nintendo não tem tanta influência direta. 

O Wii, na verdade é o sétimo console mais vendido de todos os tempos, enquanto seus competidores de geração, Play Station 3 e Xbox 360 ocupam a oitava e nona posição respectivamente. 

A febre que foi o console se deve também pela inovação que trouxe os famosos controles de movimento. (Reação a Incentivos)

Em números brutos, o GameBoy, Switch e DS, também da Nintendo, venderam mais que o Wii. Apesar disso, a venda deles não foi afetada negativamente no Japão e entram no grupo dos 7 afetados positivamente no país. Nesse caso, a febre global parece ter crescido juntamente com a situação nacional do console.

### Reflexão de Mercado

O mercado de consoles aparenta estar entre um oligopólio e uma concorrência monopolística. A entrada e saída não é livre, dada a dificuldade de produção de um console e o custo de consolidação no mercado. Apesar disso, os produtos são diferenciáveis, cada um oferecendo sua própria inovação e vantagens, como podemos ver no caso do Wii, que foge da regra até dentro de sua própria empresa.

Com essas análises, também fica mais fácil de prever onde o marketing de um produto tende a gerar mais resultado. É praticamente inútil para a Microsoft anunciar seus produtos no Japão. O país afeta tão negativamente suas vendas que o custo de se firmar no mercado de videogames lá pode ultrapassar o possível retorno desse investimento.

A análise também pode apresentar características do mercado local que podem passar despercebidas. Um exemplo é o aparente forte nacionalismo japonês quanto as suas empresas, preferindo comprar produtos nacionais a estrangeiros.